In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!/usr/bin/python

import sys
import pickle
sys.path.append("../tools/")

import pandas as pd
import numpy as np
import plotly.plotly as py
import cufflinks as cf
import plotly.graph_objs as go
from plotly.graph_objs import *

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
from sklearn import svm
from sklearn.metrics import accuracy_score, recall_score
from sklearn.model_selection import train_test_split
from time import time
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import AdaBoostClassifier
from tester import test_classifier

warnings.filterwarnings('ignore')

C:\Users\agdje\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [3]:
### Load the dictionary containing the dataset
data_dict = pickle.load(
    open("../final_project/final_project_dataset.pkl", "r"))

In [4]:
#Select what features to use:

#all in USD, features ordered like financial pdf:
financial_features = [
    'salary', 'bonus', 'long_term_incentive', 'deferred_income',
    'deferral_payments', 'loan_advances', 'other', 'expenses', 'director_fees',
    'total_payments', 'exercised_stock_options', 'restricted_stock',
    'restricted_stock_deferred', 'total_stock_value'
]
#(units are generally number of emails messages; notable exception is ‘email_address’, which is a text string)
email_features = [
    'to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages',
    'from_this_person_to_poi', 'shared_receipt_with_poi'
]

#for now, use all.  Let's trim later.
features_list = financial_features + email_features + ['poi']

In [5]:
#Move data into pandas dataframe
#Cribbed heavily from Miles: https://discussions.udacity.com/t/pickling-pandas-df/174753
df = pd.DataFrame.from_records(list(data_dict.values()))
employees = pd.Series(list(data_dict.keys()))
df.columns.values
# __, cols = df.shape
# print cols

array(['bonus', 'deferral_payments', 'deferred_income', 'director_fees',
       'email_address', 'exercised_stock_options', 'expenses',
       'from_messages', 'from_poi_to_this_person',
       'from_this_person_to_poi', 'loan_advances', 'long_term_incentive',
       'other', 'poi', 'restricted_stock', 'restricted_stock_deferred',
       'salary', 'shared_receipt_with_poi', 'to_messages',
       'total_payments', 'total_stock_value'], dtype=object)

In [6]:
#The data set is using the string 'NaN' instead of nan, which doesn't play well with describe
df = df.replace('NaN', np.nan)

df.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,8.200000e+01,3.900000e+01,4.900000e+01,1.700000e+01,1.020000e+02,9.500000e+01,86.000000,86.000000,86.000000,4.000000e+00,6.600000e+01,9.300000e+01,1.100000e+02,1.800000e+01,9.500000e+01,86.000000,86.000000,1.250000e+02,1.260000e+02
mean,2.374235e+06,1.642674e+06,-1.140475e+06,1.668049e+05,5.987054e+06,1.087289e+05,608.790698,64.895349,41.232558,4.196250e+07,1.470361e+06,9.190650e+05,2.321741e+06,1.664106e+05,5.621943e+05,1176.465116,2073.860465,5.081526e+06,6.773957e+06
std,1.071333e+07,5.161930e+06,4.025406e+06,3.198914e+05,3.106201e+07,5.335348e+05,1841.033949,86.979244,100.073111,4.708321e+07,5.942759e+06,4.589253e+06,1.251828e+07,4.201494e+06,2.716369e+06,1178.317641,2582.700981,2.906172e+07,3.895777e+07
min,7.000000e+04,-1.025000e+05,-2.799289e+07,3.285000e+03,3.285000e+03,1.480000e+02,12.000000,0.000000,0.000000,4.000000e+05,6.922300e+04,2.000000e+00,-2.604490e+06,-7.576788e+06,4.770000e+02,2.000000,57.000000,1.480000e+02,-4.409300e+04
25%,4.312500e+05,8.157300e+04,-6.948620e+05,9.878400e+04,5.278862e+05,2.261400e+04,22.750000,10.000000,1.000000,1.600000e+06,2.812500e+05,1.215000e+03,2.540180e+05,-3.896218e+05,2.118160e+05,249.750000,541.250000,3.944750e+05,4.945102e+05
50%,7.693750e+05,2.274490e+05,-1.597920e+05,1.085790e+05,1.310814e+06,4.695000e+04,41.000000,35.000000,8.000000,4.176250e+07,4.420350e+05,5.238200e+04,4.517400e+05,-1.469750e+05,2.599960e+05,740.500000,1211.000000,1.101393e+06,1.102872e+06
75%,1.200000e+06,1.002672e+06,-3.834600e+04,1.137840e+05,2.547724e+06,7.995250e+04,145.500000,72.250000,24.750000,8.212500e+07,9.386720e+05,3.620960e+05,1.002370e+06,-7.500975e+04,3.121170e+05,1888.250000,2634.750000,2.093263e+06,2.949847e+06
max,9.734362e+07,3.208340e+07,-8.330000e+02,1.398517e+06,3.117640e+08,5.235198e+06,14368.000000,528.000000,609.000000,8.392500e+07,4.852193e+07,4.266759e+07,1.303223e+08,1.545629e+07,2.670423e+07,5521.000000,15149.000000,3.098866e+08,4.345095e+08


In [7]:
persons_of_interest = [k for k in data_dict if data_dict[k]['poi']]

#known issue in this data -- email list is restricted to persons of interest who worked for Enron
poi_names = pd.read_csv("../final_project/poi_names.txt", "\s+")
print 'named persons of interest: {0}'.format(len(poi_names))

print(
    'There are {0} persons in the data, {1} or {2:.2f}% are persons of interest.'
).format(len(df), len(persons_of_interest), 100. * len(persons_of_interest) / len(df))

print ('In other words, there are {0} persons total, {1} persons of interest, and {2} who are not persons of interest.'
).format(len(df), len(persons_of_interest), (len(df) - len(persons_of_interest)))

named persons of interest: 35
There are 146 persons in the data, 18 or 12.33% are persons of interest.
In other words, there are 146 persons total, 18 persons of interest, and 128 who are not persons of interest.


In [8]:
print 'Where is data missing?  What values have NaN, and what percentage of those is missing data?'
for col in df.columns.values:
    isnan = df[col].isnull().sum()
    print '{0} NaN values: {1} people = {2:.2f}%'.format(col, isnan, 100. *
                                                         isnan / len(df))

Where is data missing?  What values have NaN, and what percentage of those is missing data?
bonus NaN values: 64 people = 43.84%
deferral_payments NaN values: 107 people = 73.29%
deferred_income NaN values: 97 people = 66.44%
director_fees NaN values: 129 people = 88.36%
email_address NaN values: 35 people = 23.97%
exercised_stock_options NaN values: 44 people = 30.14%
expenses NaN values: 51 people = 34.93%
from_messages NaN values: 60 people = 41.10%
from_poi_to_this_person NaN values: 60 people = 41.10%
from_this_person_to_poi NaN values: 60 people = 41.10%
loan_advances NaN values: 142 people = 97.26%
long_term_incentive NaN values: 80 people = 54.79%
other NaN values: 53 people = 36.30%
poi NaN values: 0 people = 0.00%
restricted_stock NaN values: 36 people = 24.66%
restricted_stock_deferred NaN values: 128 people = 87.67%
salary NaN values: 51 people = 34.93%
shared_receipt_with_poi NaN values: 60 people = 41.10%
to_messages NaN values: 60 people = 41.10%
total_payments NaN value

In [9]:
#Add the employee names into the pandas dataframe
df = pd.concat([employees, df], axis=1)
df = df.rename(index=str, columns={0: "employee_name"})

df

,employee_name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,METTS MARK,600000.0,NaN,NaN,NaN,mark.metts@enron.com,NaN,94299.0,29.0,38.0,...,NaN,1740.0,False,585062.0,NaN,365788.0,702.0,807.0,1061827.0,585062.0
1,BAXTER JOHN C,1200000.0,1295738.0,-1386055.0,NaN,NaN,6680544.0,11200.0,NaN,NaN,...,1586055.0,2660303.0,False,3942714.0,NaN,267102.0,NaN,NaN,5634343.0,10623258.0
2,ELLIOTT STEVEN,350000.0,NaN,-400729.0,NaN,steven.elliott@enron.com,4890344.0,78552.0,NaN,NaN,...,NaN,12961.0,False,1788391.0,NaN,170941.0,NaN,NaN,211725.0,6678735.0
3,CORDES WILLIAM R,NaN,NaN,NaN,NaN,bill.cordes@enron.com,651850.0,NaN,12.0,10.0,...,NaN,NaN,False,386335.0,NaN,NaN,58.0,764.0,NaN,1038185.0
4,HANNON KEVIN P,1500000.0,NaN,-3117011.0,NaN,kevin.hannon@enron.com,5538001.0,34039.0,32.0,32.0,...,1617011.0,11350.0,True,853064.0,NaN,243293.0,1035.0,1045.0,288682.0,6391065.0
5,MORDAUNT KRISTINA M,325000.0,NaN,NaN,NaN,kristina.mordaunt@enron.com,NaN,35018.0,NaN,NaN,...,NaN,1411.0,False,208510.0,NaN,267093.0,NaN,NaN,628522.0,208510.0
6,MEYER ROCKFORD G,NaN,1848227.0,NaN,NaN,rockford.meyer@enron.com,493489.0,NaN,28.0,0.0,...,NaN,NaN,False,462384.0,NaN,NaN,22.0,232.0,1848227.0,955873.0
7,MCMAHON JEFFREY,2600000.0,NaN,NaN,NaN,jeffrey.mcmahon@enron.com,1104054.0,137108.0,48.0,58.0,...,694862.0,297353.0,False,558801.0,NaN,370448.0,2228.0,2355.0,4099771.0,1662855.0
8,HORTON STANLEY C,NaN,3131860.0,NaN,NaN,stanley.horton@enron.com,5210569.0,NaN,1073.0,44.0,...,NaN,NaN,False,2046079.0,NaN,NaN,1074.0,2350.0,3131860.0,7256648.0
9,PIPER GREGORY F,400000.0,1130036.0,-33333.0,NaN,greg.piper@enron.com,880290.0,43057.0,222.0,61.0,...,NaN,778.0,False,409554.0,-409554.0,197091.0,742.0,1238.0,1737629.0,880290.0


In [10]:
#Let's see how compensation 'balances' horizontally, so we know which stock options have the richest information.
for col in df.columns.values:
    if 'stock' in col:
        print '{0} total is: {1}'.format(col, df[col].sum())
print ''
print 'total_stock_value should be the sum of the other stock options, as it\'s labeled total. Let\'s see if that holds true.'
print ''
print 'The balanced stock data is: {0}, we would expect zero.'.format(
    (df['total_stock_value'].sum() - df['exercised_stock_options'].sum() -
     (df['restricted_stock'].sum() + df['restricted_stock_deferred'].sum())))

exercised_stock_options total is: 610679485.0
restricted_stock total is: 255391525.0
restricted_stock_deferred total is: 2995390.0
total_stock_value total is: 853518639.0

total_stock_value should be the sum of the other stock options, as it's labeled total. Let's see if that holds true.

The balanced stock data is: -15547761.0, we would expect zero.


In [11]:
#Who in the data has a line that isn't balancing?
df.loc[df['total_stock_value'].fillna(0) -
       df['exercised_stock_options'].fillna(0) -
       (df['restricted_stock'].fillna(0) +
        df['restricted_stock_deferred'].fillna(0)) != 0]

,employee_name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
24,BELFER ROBERT,NaN,-102500.0,NaN,3285.0,NaN,3285.0,NaN,NaN,NaN,...,NaN,NaN,False,NaN,44093.0,NaN,NaN,NaN,102500.0,-44093.0
118,BHATNAGAR SANJAY,NaN,NaN,NaN,137864.0,sanjay.bhatnagar@enron.com,2604490.0,NaN,29.0,0.0,...,NaN,137864.0,False,-2604490.0,15456290.0,NaN,463.0,523.0,15456290.0,NaN


In [12]:
#After a visual check, these two people are NOT reflecting what's in the FindLaw PDF of payments to insiders.
print 'Semi-manually adjusting financials for {0}'.format(employees[24])
#24 is BELFER ROBERT, his payments are all shifted to the right one starting at deferred_income.
for col in financial_features[3:-1]:
    col_loc = df.columns.get_loc(col)
    next_col_loc = df.columns.get_loc(financial_features[
        financial_features.index(col) + 1])
    #     print col_loc, df.iloc[24,col_loc], df.iloc[24,next_col_loc]
    df.iloc[24, col_loc] = df.iloc[24, next_col_loc]

df.iloc[24, df.columns.get_loc('total_stock_value')] = np.nan

Semi-manually adjusting financials for BELFER ROBERT


In [13]:
print 'Semi-manually adjusting financials for {0}'.format(employees[118])

#Sanjay's values all need to slide one to the right after 'Other', which was missing a - in the pdf.
#Loan Advances is already NaN, so we can use that value to fill other.
for col in reversed(financial_features[6:]):
    col_loc = df.columns.get_loc(col)
    prev_col_loc = df.columns.get_loc(financial_features[
        financial_features.index(col) - 1])
    #     print df.columns[col_loc], df.iloc[118,col_loc], df.iloc[118,prev_col_loc]
    df.iloc[118, col_loc] = df.iloc[118, prev_col_loc]

# print df.iloc[118]

Semi-manually adjusting financials for BHATNAGAR SANJAY


In [14]:
#Check the stock balancing now that we've adjusted some figures:
for col in df.columns.values:
    if 'stock' in col:
        print '{0} total is: {1}'.format(col, df[col].sum())
print ''
print 'total_stock_value should be the sum of the other stock options, as it\'s labeled total. Let\'s see if that holds true.'
print ''
print 'The balanced stock data is now: {0}, we would expect zero.'.format(
    (df['total_stock_value'].sum() - df['exercised_stock_options'].sum() -
     (df['restricted_stock'].sum() + df['restricted_stock_deferred'].sum())))

exercised_stock_options total is: 623528000.0
restricted_stock total is: 260644598.0
restricted_stock_deferred total is: -15153576.0
total_stock_value total is: 869019022.0

total_stock_value should be the sum of the other stock options, as it's labeled total. Let's see if that holds true.

The balanced stock data is now: 0.0, we would expect zero.


Clean up outliers

In [15]:
#Let's do the same check for total payments:
print 'The balanced payments data is: {0}, we would expect zero.'.format(
    (df['total_payments'].sum() - df['salary'].sum() - df['bonus'].sum() -
     df['long_term_incentive'].sum() - df['deferred_income'].sum() -
     df['deferral_payments'].sum() - df['loan_advances'].sum() -
     df['other'].sum() - df['expenses'].sum() - df['director_fees'].sum()))

The balanced payments data is: 0.0, we would expect zero.


In [16]:
#Because the data was gathered as part of a lawsuit, we can safely assume that any NaN values in the financial fields had no related payments.
for col in financial_features:
    df[col] = df[col].fillna(0)

In [17]:
#Quick visual check for outliers
df[['salary', 'bonus']].iplot(
    kind='scatter',
    mode='markers',
    x='salary',
    y='bonus',
    filename='cufflinks/simple-scatter')

In [18]:
outlier_salary_bonus = {
    k: v
    for k, v in data_dict.items()
    if v['bonus'] >= 90000000 and v['bonus'] <> 'NaN' and v['salary'] >=
    26000000
}

outlier_salary_bonus

{'TOTAL': {'bonus': 97343619,
  'deferral_payments': 32083396,
  'deferred_income': -27992891,
  'director_fees': 1398517,
  'email_address': 'NaN',
  'exercised_stock_options': 311764000,
  'expenses': 5235198,
  'from_messages': 'NaN',
  'from_poi_to_this_person': 'NaN',
  'from_this_person_to_poi': 'NaN',
  'loan_advances': 83925000,
  'long_term_incentive': 48521928,
  'other': 42667589,
  'poi': False,
  'restricted_stock': 130322299,
  'restricted_stock_deferred': -7576788,
  'salary': 26704229,
  'shared_receipt_with_poi': 'NaN',
  'to_messages': 'NaN',
  'total_payments': 309886585,
  'total_stock_value': 434509511}}

In [19]:
#get the total line out of _everything_.
data_dict.pop('TOTAL', 0)

total_idx = employees[employees == 'TOTAL'].index[0]
df = df.drop(df.index[total_idx])

employees = employees.drop(employees[employees == 'TOTAL'].index[0])

In [20]:
#Now that we've seen a "total" record, let's manually check names to see who else we have present:
for e in employees:
    print e

METTS MARK
BAXTER JOHN C
ELLIOTT STEVEN
CORDES WILLIAM R
HANNON KEVIN P
MORDAUNT KRISTINA M
MEYER ROCKFORD G
MCMAHON JEFFREY
HORTON STANLEY C
PIPER GREGORY F
HUMPHREY GENE E
UMANOFF ADAM S
BLACHMAN JEREMY M
SUNDE MARTIN
GIBBS DANA R
LOWRY CHARLES P
COLWELL WESLEY
MULLER MARK S
JACKSON CHARLENE R
WESTFAHL RICHARD K
WALTERS GARETH W
WALLS JR ROBERT H
KITCHEN LOUISE
CHAN RONNIE
BELFER ROBERT
SHANKMAN JEFFREY A
WODRASKA JOHN
BERGSIEKER RICHARD P
URQUHART JOHN A
BIBI PHILIPPE A
RIEKER PAULA H
WHALEY DAVID A
BECK SALLY W
HAUG DAVID L
ECHOLS JOHN B
MENDELSOHN JOHN
HICKERSON GARY J
CLINE KENNETH W
LEWIS RICHARD
HAYES ROBERT E
MCCARTY DANNY J
KOPPER MICHAEL J
LEFF DANIEL P
LAVORATO JOHN J
BERBERIAN DAVID
DETMERING TIMOTHY J
WAKEHAM JOHN
POWERS WILLIAM
GOLD JOSEPH
BANNANTINE JAMES M
DUNCAN JOHN H
SHAPIRO RICHARD S
SHERRIFF JOHN R
SHELBY REX
LEMAISTRE CHARLES
DEFFNER JOSEPH M
KISHKILL JOSEPH G
WHALLEY LAWRENCE G
MCCONNELL MICHAEL S
PIRO JIM
DELAINEY DAVID W
SULLIVAN-SHAKLOVITZ COLLEEN
WROBEL BRUC

In [21]:
#"THE TRAVEL AGENCY IN THE PARK" is clearly not a person.  Let's drop it.
data_dict.pop('THE TRAVEL AGENCY IN THE PARK', 0)

total_idx = employees[employees == 'THE TRAVEL AGENCY IN THE PARK'].index[0]
df = df.drop(df.index[total_idx])

employees = employees.drop(employees[employees ==
                                     'THE TRAVEL AGENCY IN THE PARK'].index[0])

In [22]:
#Re-run the visual check for outliers
df[['salary', 'bonus']].iplot(
    kind='scatter',
    mode='markers',
    x='salary',
    y='bonus',
    filename='cufflinks/simple-scatter')

In [23]:
#We have a person at 0,0?  
zero_point = {
    k: v
    for k, v in data_dict.items()
    if all(i == 'NaN' or i == 0 for i in v.values())
}

zero_point

{'LOCKHART EUGENE E': {'bonus': 'NaN',
  'deferral_payments': 'NaN',
  'deferred_income': 'NaN',
  'director_fees': 'NaN',
  'email_address': 'NaN',
  'exercised_stock_options': 'NaN',
  'expenses': 'NaN',
  'from_messages': 'NaN',
  'from_poi_to_this_person': 'NaN',
  'from_this_person_to_poi': 'NaN',
  'loan_advances': 'NaN',
  'long_term_incentive': 'NaN',
  'other': 'NaN',
  'poi': False,
  'restricted_stock': 'NaN',
  'restricted_stock_deferred': 'NaN',
  'salary': 'NaN',
  'shared_receipt_with_poi': 'NaN',
  'to_messages': 'NaN',
  'total_payments': 'NaN',
  'total_stock_value': 'NaN'}}

In [24]:
#"Eugene Lockhart" has no values.  Dropping.
data_dict.pop('LOCKHART EUGENE E', 0)

total_idx = employees[employees == 'LOCKHART EUGENE E'].index[0]
df = df.drop(df.index[total_idx])

employees = employees.drop(employees[employees == 'LOCKHART EUGENE E'].index[
    0])

In [25]:
#Add a new column, total_compensation, to summarize the overall compensation someone's getting from the company.
#this is not scaled! Stock is a smaller proportion of compensation and I do not believe it should be weighed as 'more' of the package.
df['total_compensation'] = df['total_payments'].fillna(0) + df[
    'total_stock_value'].fillna(0)

Let's follow the money -- were people who were significantly higher funded more likely to be persons of interest?

In [26]:
#Fun with Graphs: total_payments+total_stock_value with Employee Names included
# df[['total_payments','total_stock_value', 'employee_name', 'poi']].iplot(kind='scatter', mode='markers', x='total_payments', y='total_stock_value',
#                                                                   text= 'poi', filename='cufflinks/simple-scatter')

trace = go.Scatter(
    x=df.total_payments,
    y=df.total_stock_value,
    mode='markers',
    marker=dict(
        size='16',
        color=map(lambda x: 1 if x else 0, df.poi),
        colorscale='Viridis',
        showscale=True),
    text=df.employee_name, )
# layout = Layout(
#     xaxis = dict(title='total payments'),
#     yaxis = dict(title='total stock value'))
py.iplot([trace], filename='scatter-plot-with-colorscale')

In [27]:
#What if it's just salary and bonus?

trace = go.Scatter(
    x=df.salary,
    y=df.bonus,
    mode='markers',
    marker=dict(
        size='12',
        color=map(lambda x: 1 if x else 0, df.poi),
        colorscale='Viridis',
        showscale=True),
    text=df.employee_name, )
py.iplot([trace], filename='scatter-plot-with-colorscale')

In [28]:
#simple/beautiful outlier detection function from https://github.com/joferkington/oost_paper_code/blob/master/utilities.py
#input is a list and an optional z-score, output is a True/False for whether that specific value would be above the z-score.
def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:, None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [29]:
#Let's mark the people who are extremely well paid
df['total_comp_outlier'] = is_outlier(df.total_compensation)

In [30]:
#visual check of the z-score method
trace = go.Scatter(
    x=df.total_payments,
    y=df.total_stock_value,
    mode='markers',
    marker=dict(
        size='12',
        color=map(lambda x: 1 if x else 0, is_outlier(df.total_compensation)),
        colorscale='Viridis',
        showscale=True),
    text=df.employee_name, )
py.iplot([trace], filename='scatter-plot-with-colorscale')

In [31]:
#Quick check of how many persons of interest were and weren't in our exceptionally well-paid category
# df.groupby(['poi', 'total_comp_outlier']).size()
df[['poi', 'total_comp_outlier', 'employee_name']].groupby(
    ['poi', 'total_comp_outlier']).agg(['count'])

employee_name
                                 count
poi   total_comp_outlier              
False False                        119
      True                           6
True  False                         13
      True                           5

In [32]:
#convert the dataframe back into a dictionary
my_dataset = df.to_dict('index')

In [33]:
# #First, we'll do a basic check to see how much overall compensation could predict whether someone's a poi
# #based on the outlier correlation we saw above, I'm assuming it's low
features_list = ['poi', 'total_payments', 'total_stock_value']


def create_test_split(features_list):
    data = featureFormat(my_dataset, features_list)
    global labels, features
    labels, features = targetFeatureSplit(data)
    global features_train, features_test, labels_train, labels_test
    features_train, features_test, labels_train, labels_test = \
        train_test_split(features, labels, random_state=42, test_size=0.25)


create_test_split(features_list)


def classify_SVC(f_train, l_train, f_test, l_test):
    clf = svm.SVC(kernel='rbf', C=10000)
    t0 = time()

    # features_train = features_train[:len(features_train)/100] 
    # labels_train = labels_train[:len(labels_train)/100] 

    clf.fit(f_train, l_train)
    print "training time:", round(time() - t0, 3), "s"

    t0 = time()
    pred = clf.predict(f_test)
    print pred
    print "prediction time:", round(time() - t0, 3), "s"

    t0 = time()
    print accuracy_score(l_test, pred)
    print "accuracy time:", round(time() - t0, 3), "s"

    t0 = time()
    print recall_score(l_test, pred)
    print "recall time:", round(time() - t0, 3), "s"

    cm = confusion_matrix(labels_test, pred)
    print(
        'There are {0} True Negatives, {1} False Positives, {2} False Negatives, and {3} True Positives'
    ).format(cm[0][0], cm[0][1], cm[1][0], cm[1][1])


classify_SVC(features_train, labels_train, features_test, labels_test)

training time: 0.001 s
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
prediction time: 0.002 s
0.944444444444
accuracy time: 0.0 s
0.0
recall time: 0.001 s
There are 34 True Negatives, 0 False Positives, 2 False Negatives, and 0 True Positives


In [34]:
# #TODO: Consider making graph of actual y/n poi on left, predicted y/n poi on right

Based on total payments and total stock value alone, we can determine the chance of someone being a person of interest with a reasonable accuracy.  While the prediction numbers look good at face value, this will not do as this model is predicting _everyone_ to not be a person of interest.  This result has far too many Type II errors to be a good indicator of who to investigate.

In [35]:
#How often were persons of interest emailing each other?
trace = go.Scatter(
    x=df.from_poi_to_this_person,
    y=df.from_this_person_to_poi,
    mode='markers',
    marker=dict(
        size='12',
        color=map(lambda x: 1 if x else 0, df.poi),
        colorscale='Viridis',
        showscale=True),
    text=df.employee_name, )
py.iplot([trace], filename='scatter-plot-with-colorscale')

In [36]:
scaler = MinMaxScaler()
#Assuming if we have no emails to a person in the records, there are none, as this data was obtained through legal discovery.
df['from_poi_to_this_person'] = df['from_poi_to_this_person'].fillna(0)
df['from_this_person_to_poi'] = df['from_this_person_to_poi'].fillna(0)
df['from_poi_scaled'] = df['from_poi_to_this_person']
df['to_poi_scaled'] = df['from_this_person_to_poi']

df[['from_poi_scaled', 'to_poi_scaled']] = scaler.fit_transform(df[
    ['from_poi_to_this_person', 'from_this_person_to_poi']])

In [37]:
#How often were persons of interest emailing each other?
trace = go.Scatter(
    x=df.from_poi_scaled,
    y=df.to_poi_scaled,
    mode='markers',
    marker=dict(
        size='12',
        color=map(lambda x: 1 if x else 0, df.poi),
        colorscale='Viridis',
        showscale=True),
    text=df.employee_name, )
py.iplot([trace], filename='scatter-plot-with-colorscale')

In [38]:
#That was interesting to see, but shouldn't be used for testing -- we don't want to scale based on all data points
#then train PCA on some data points.
df = df.drop(['from_poi_scaled'], axis=1)
df = df.drop(['to_poi_scaled'], axis=1)

In [39]:
#convert the dataframe back into a dictionary
my_dataset = df.to_dict('index')

In [40]:
features_list = ['poi', 'from_poi_to_this_person', 'from_this_person_to_poi']
create_test_split(features_list)

In [41]:
features_train_scaled = scaler.fit_transform(features_train)
features_test_scaled = scaler.transform(features_test)

In [42]:
#create & fit PCA
from sklearn.decomposition import PCA
t0 = time()
pca = PCA(n_components=2).fit(features_train_scaled)
print "done in %0.3fs" % (time() - t0)

done in 0.002s


In [43]:
features_train_transformed = pca.transform(features_train_scaled)
features_test_transformed = pca.transform(features_test_scaled)

In [44]:
print('Predict without scaling or PCA')
classify_SVC(features_train, labels_train, features_test, labels_test)

Predict without scaling or PCA
training time: 0.001 s
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
prediction time: 0.001 s
0.85
accuracy time: 0.0 s
0.0
recall time: 0.001 s
There are 17 True Negatives, 0 False Positives, 3 False Negatives, and 0 True Positives


In [45]:
print('Predict with scaling only')
classify_SVC(features_train_scaled, labels_train, features_test_scaled,
             labels_test)

Predict with scaling only
training time: 0.002 s
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
prediction time: 0.001 s
0.8
accuracy time: 0.0 s
0.0
recall time: 0.002 s
There are 16 True Negatives, 1 False Positives, 3 False Negatives, and 0 True Positives


In [46]:
print('Predict with scaling and PCA')
classify_SVC(features_train_transformed, labels_train,
             features_test_transformed, labels_test)

Predict with scaling and PCA
training time: 0.003 s
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
prediction time: 0.001 s
0.8
accuracy time: 0.0 s
0.0
recall time: 0.001 s
There are 16 True Negatives, 1 False Positives, 3 False Negatives, and 0 True Positives


Again, based on the number of emails going between persons and known persons of interest, we cannot predict whether someone would be a person of interest with enough certainty.  I'd still like more false positives than false negatives, as we'd like to know who else to investigate when someone is suspect.

Which brings us to my fleshed-out question -- were persons of interest better connected than non-POIs?  
I'm specifically interested in how often persons of interest emailed the higher-paid staff at Enron vs non-POIs.

In [47]:
features_list = [
    'poi', 'total_payments', 'total_stock_value', 'from_poi_to_this_person',
    'from_this_person_to_poi'
]
create_test_split(features_list)

In [48]:
print('Predict using total payments, total stock, from poi, to poi')
classify_SVC(features_train, labels_train, features_test, labels_test)

Predict using total payments, total stock, from poi, to poi
training time: 0.001 s
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
prediction time: 0.001 s
0.944444444444
accuracy time: 0.0 s
0.0
recall time: 0.001 s
There are 34 True Negatives, 0 False Positives, 2 False Negatives, and 0 True Positives


That's even worse.  Clearly we cannot use just these values to predict who is a POI.

In [49]:
#Let's try selecting features based on the financial features and connectivity / how often the person sent poi emails
features_list = ['poi'] + financial_features + [
    'from_poi_to_this_person', 'from_this_person_to_poi'
]
create_test_split(features_list)

kbest = SelectKBest(f_classif, k=10)
selected_features = kbest.fit_transform(features_train, labels_train)

In [50]:
features_selected = [
    features_list[i + 1] for i in kbest.get_support(indices=True)
]

print 'Features selected by SelectKBest:'
print features_selected

Features selected by SelectKBest:
['salary', 'bonus', 'deferred_income', 'loan_advances', 'expenses', 'total_payments', 'exercised_stock_options', 'restricted_stock', 'total_stock_value', 'from_poi_to_this_person']


SelectKBest is suggesting we disregard email frequency to POIs in our predictions.

In [51]:
print 'Looking at the f score for each to validate that our selected scores had significantly higher scores than non-selected'
print [i for i in kbest.get_support(indices=True)]
kbest.scores_

Looking at the f score for each to validate that our selected scores had significantly higher scores than non-selected
[0, 1, 3, 5, 7, 9, 10, 11, 13, 14]


array([  9.05321627,  13.31909446,   5.22333556,  13.0323781 ,
         0.1554117 ,   5.45507033,   3.53703896,   6.00885032,
         2.21307488,   6.16144495,  12.40878064,   8.97050543,
         0.83774729,  13.66940293,   5.56606039,   3.00002746])

In [52]:
create_test_split(features_list)
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=0)

# svc = svm.SVC()
pipe = make_pipeline(MinMaxScaler(), PCA(), LinearSVC(), GradientBoostingClassifier())

parameters = {
    'pca__n_components': range(1,16),
    'pca__svd_solver': ['auto', 'full', 'arpack', 'randomized'],
    'linearsvc__C': range(1,100)
}

gs = GridSearchCV(pipe, parameters, cv=cv, scoring='f1')

gs.fit(features, labels)
print("The best parameters are: {0}").format(gs.best_params_)

clf = gs.best_estimator_

The best parameters are: {'linearsvc__C': 37, 'pca__n_components': 13, 'pca__svd_solver': 'arpack'}


In [53]:
print test_classifier(clf, my_dataset, features_list)

Pipeline(steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('pca', PCA(copy=True, iterated_power='auto', n_components=13, random_state=None,
  svd_solver='arpack', tol=0.0, whiten=False)), ('linearsvc', LinearSVC(C=37, class_weight=None, dual=True, fit_intercept=True,
     intercept_sca...=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False))])
	Accuracy: 0.83193	Precision: 0.30888	Recall: 0.21050	F1: 0.25037	F2: 0.22482
	Total predictions: 15000	True positives:  421	False positives:  942	False negatives: 1579	True negatives: 12058

None


In [54]:
create_test_split(features_list)
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=0)

# svc = svm.SVC()
pipe = make_pipeline(MinMaxScaler(), PCA(), LinearSVC(), GradientBoostingClassifier())

parameters = {
    'pca__n_components': range(1,16),
    'pca__svd_solver': ['auto', 'full', 'arpack', 'randomized'],
    'linearsvc__C': range(1,100)
}

gs = GridSearchCV(pipe, parameters, cv=cv, scoring='recall')

gs.fit(features, labels)
print("The best parameters are: {0}").format(gs.best_params_)

clf = gs.best_estimator_

The best parameters are: {'linearsvc__C': 5, 'pca__n_components': 13, 'pca__svd_solver': 'full'}


In [55]:
print test_classifier(clf, my_dataset, features_list)

Pipeline(steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('pca', PCA(copy=True, iterated_power='auto', n_components=13, random_state=None,
  svd_solver='full', tol=0.0, whiten=False)), ('linearsvc', LinearSVC(C=5, class_weight=None, dual=True, fit_intercept=True,
     intercept_scalin...=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False))])
	Accuracy: 0.83233	Precision: 0.31327	Recall: 0.21600	F1: 0.25570	F2: 0.23030
	Total predictions: 15000	True positives:  432	False positives:  947	False negatives: 1568	True negatives: 12053

None


In [56]:
create_test_split(features_list)
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=0)

pipe = make_pipeline(SelectKBest(), AdaBoostClassifier())

parameters = {
    'selectkbest__k': range(1, 15),
    'adaboostclassifier__n_estimators':
    [15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70],
    'adaboostclassifier__learning_rate': np.arange(0.1, 1, 0.1)
}

gs = GridSearchCV(pipe, parameters, cv=cv, scoring='recall')

gs.fit(features, labels)
print("The best parameters are: {0}").format(gs.best_params_)

clf = gs.best_estimator_

The best parameters are: {'selectkbest__k': 12, 'adaboostclassifier__n_estimators': 30, 'adaboostclassifier__learning_rate': 0.70000000000000007}


In [57]:
print test_classifier(clf, my_dataset, features_list)

Pipeline(steps=[('selectkbest', SelectKBest(k=12, score_func=<function f_classif at 0x000000000DBABE48>)), ('adaboostclassifier', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.70000000000000007, n_estimators=30,
          random_state=None))])
	Accuracy: 0.85080	Precision: 0.42077	Recall: 0.31600	F1: 0.36094	F2: 0.33256
	Total predictions: 15000	True positives:  632	False positives:  870	False negatives: 1368	True negatives: 12130

None


In [58]:
#Add scaling
create_test_split(features_list)
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=0)

pipe = make_pipeline(MinMaxScaler(), SelectKBest(), AdaBoostClassifier())

parameters = {
    'selectkbest__k': range(1,15),
    'adaboostclassifier__n_estimators': [15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70],
    'adaboostclassifier__learning_rate': np.arange(0.1,1,0.1)
}

gs = GridSearchCV(pipe, parameters, cv=cv, scoring='recall')

gs.fit(features, labels)
print("The best parameters are: {0}").format(gs.best_params_)

clf = gs.best_estimator_

The best parameters are: {'selectkbest__k': 12, 'adaboostclassifier__n_estimators': 30, 'adaboostclassifier__learning_rate': 0.70000000000000007}


In [59]:
clf

Pipeline(steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selectkbest', SelectKBest(k=12, score_func=<function f_classif at 0x000000000DBABE48>)), ('adaboostclassifier', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.70000000000000007, n_estimators=30,
          random_state=None))])

In [60]:
features_selected = [
    features_list[i + 1]
    for i in clf.named_steps['selectkbest'].get_support(indices=True)
]

print 'Features selected by SelectKBest:'
print features_selected

Features selected by SelectKBest:
['salary', 'bonus', 'long_term_incentive', 'deferred_income', 'loan_advances', 'other', 'expenses', 'total_payments', 'exercised_stock_options', 'restricted_stock', 'total_stock_value', 'from_poi_to_this_person']


In [61]:
print 'Looking at the f score for each to validate that our selected scores had significantly higher scores than non-selected'
print [i for i in clf.named_steps['selectkbest'].get_support(indices=True)]
clf.named_steps['selectkbest'].scores_

Looking at the f score for each to validate that our selected scores had significantly higher scores than non-selected
[0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14]


array([ 18.28968404,  20.79225205,   9.92218601,  11.42489149,
         0.22885962,   7.18405566,   4.2024363 ,   5.41890019,
         2.13148399,   9.28387362,  22.34897541,   8.82544222,
         0.76814634,  22.51054909,   5.24344971,   2.38261211])

In [62]:
print test_classifier(clf, my_dataset, features_list)

Pipeline(steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selectkbest', SelectKBest(k=12, score_func=<function f_classif at 0x000000000DBABE48>)), ('adaboostclassifier', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.70000000000000007, n_estimators=30,
          random_state=None))])
	Accuracy: 0.85147	Precision: 0.42369	Recall: 0.31650	F1: 0.36234	F2: 0.33337
	Total predictions: 15000	True positives:  633	False positives:  861	False negatives: 1367	True negatives: 12139

None


In [63]:
dump_classifier_and_data(clf, my_dataset, features_list)

In [64]:
#Future possibility, elephant is too large for scope right now: Consider checking for -- is there a group of words where the poi to poi emails [no non-pois in email?] had a higher usage
#than those words in gen pop